# Introduction to the command line and BASH scripting

The command line is a text interface that allows you interact efficently with your computer's operating system and achieve tasks that would be difficult or impossible using a Graphical User Interface such as Windows or OSX. A shell is a software program used to interpret commands that are input *via* a command-line interface, enabling users to interact with a computer by giving it instructions. There are a few different shells for interacting with the command line, we will be using the **Bourne Again Shell (BASH)**, for the next two labs.

Many bionformatics algorithms and software packages are **only** available as command line tools. And these are most commonly compatible with BASH. In addition to allowing you to use bioinformatics tools that would not otherwise be accessible, having a basic undertstanding of BASH will allow you to manipulate data on a large scale and stich together different bionformatics tools into pipelines that automate data processing and analysis. In these labs, for example, you will start with reads from an Illumina DNA sequencer reads and use BASH to carry out a series of processing steps with different bionformatics tools. 

You will first trim adapter sequences from your reads using a program called `AdapterRemoval`. Once you have trimmed your reads, you will assemble them into genomes using a program called `spades.py`. You will then identify genes within your assembled genomes using a program called `glimmer3`. Once you have identified genes within all of your genomes, you will extract these into a new file and map sequencing reads from and RNAseq run against the genes using a programm called `kallisto`. Mapping with Kallisto will give you a count table, in which the number of reads mapping to each gene are recorded. You will then use a statistical analysis package called `pydeseq` to identify genes that are differentially expressed between two conditions. 

**Note** I typed this in jupyter lab, so there was no spell checker. My spelling is atrocicous (see previous word for example). I would like to apologies for this in advance.

## A litte bit of background about the data you will be using.

For this lab, you will be assuming the role of a bioinformatician in a laboratory that studies the human microbiome. To make sure things runs smoothly and in a reasonable time frame, we have provided you small synthetic datasets. This is purely to avoid long running times. The software packages you will use are commonly used in many research applications, and the analysis you run could easily be scaled up to analyse real clinical data.

The data you have been given are part of a study (albiet and imaginary one) in which metagenome and RNAseq data were aquired from the gut microbiomes  of a cohort of patients. These patients were then monitored for various health outcomes over the course of ten years with the aim of identifying potenital linkages between the gut microbiome and certain disease states. You have been given RNAseq data from 12 patients. Six of these patients remained healthy over the course of the study. The other six developed colorectal cancer within two years of having their metagenome data collected. Don't worry though, all of these imaginary patients made a full recovery and then won lotto. Yay!

In addition to the RNAseq data, which is in the folders labelled ```patient_data_1``` and ```patient_data_2``` you have a folder called ```reads``` that contains sequencing reads from an exhaustive collection of gut microbes isolated from the patients. Luckily, in our imaginary scenario, there are only ten microbes in the human microbiome (not thousands). Your task, as a newly minted bioinformatician is to complete the following steps:

 1) Assemble the microbial isolate sequencing reads into genomes.
 
 2) Identify the genes in each genome and extract these to a single file.
 
 3) Map the patient RNAseq reads against the extracted gene sequences
 
 4) Use an appropriate statistical approach to identify changes in gene expression between the healthy and colorectal cancer cohorts that might provide provide insight into the connection between gut microbiome function and cancer risk
 

**A note on saving your work:** Using **ctrl+S**, or the jupyter lab dropdown menu will save your work within the VM you created. When you shut down this VM, make sure you save the machine state or you will loose your work. In addition to saving locally I **strongly reccomend** that you periodically attach the jupyter notebook you are working on to an email and send this to yourself as a fail safe. You can use firefox to login to whichever email provider you prefer and attach a file as you would normally do. The notebook is called `bash_lab.ipynb`. 

# Introductory exercises
Before we start analysing data, you will need to aquire some basic skills by completing some simple BASH scripting exercises

## Basic commands:

**Navigating the file system:** Just like a graphical interface, we can navigate through directories using the command line. The directory we are currently in is our **working directory**. The `pwd` command prints the current working directory to the console so that we know where we are. 

**Exercise 1:** In the cell below use the `pwd` command to print your working directory.

In [1]:
pwd

/Users/owenje/Documents/339_2023_reboot/labs/final_notebook_files


**Exercise 2:** In the cell below use the `cd` command to change directory to`~/339_lab-main` and list the contents of this directory using `ls`. You can put both of these commands into the same cell and gthey will be executed sequentially. 

In [4]:
cd ~/339_lab-main

In [3]:
#Run this cell to check exercise 2
ls

bash_lab.ipynb		patient_data_2.zip	reads.zip
patient_data_1.zip	python_lab.ipynb


**expected ouptut:** 
```
bash_lab.ipynb		packages_to_install.txt	python-lab.ipynb
install_conda.sh	patient_data_1.zip	reads.zip
install_packages.sh	patient_data_2.zip
```

**Note:** The tilde symbol `~` is a shortcut for your home directory. If you ever get lost, you can always find your way home by typing `cd ~`

**Note:** In BASH `*` is a wildcard that matches any pattern, so by typing `ls *.zip` you will list all files ending in the extension `*.zip`. Try it out below

In [ ]:
ls *.zip

**Unzipping files with `unzip`:** The command `unzip` will run a program that unpacks the contents of a `.zip` file. The name of the file to be unzipped is specified after the `unzip` command, with a space between the two. For example, `unzip my_file.zip` will unpack `my_file.zip`, if such a file exists in your current working directory. You might have noticed that all of the file names in this lab use underscores instead of spaces, this is because spaces are used to seperate arguments in BASH commands, which makes dealing with file names that contain spaces a pain. 

**Exercise 3:** In the cell below, write commands which will unzip all of the zip folders in your current working directory that end with the extension `.zip`

**Note:** You will need to write three lines, one for each zip file. the `unzip` command doesn't work with `*`

**Note:** If you try to unzip the same file twice, your notebook will freeze. You can fix this by restarting the kernel (using the kernel menu at the top of the page)

In [ ]:
#run this cell to check exercise 3:
ls -d */

**Expected output**
```
__MACOSX/	patient_data_1/	patient_data_2/	reads/
```

**Removing files with `rm`:** Unwanted files can be deleted using `rm`, the file(s) to be removed are specified after the `rm` command and are seperated by as space.  For example, the command `rm junk_1.txt junk_2.txt junk_3.txt` will remove three files called `junk_1.txt` `junk_2.txt` and `junk_3.txt` from your current working directory, if they exist. You can also use `*` to remove multiple files that match a given pattern. For example, `rm *junk*` would remove all files in the current directory that contain the word `junk`; `rm *.txt` would remove all files with the extension `.txt`; `rm *junk*.txt` would remove all files that contain the word `junk` *and* end in the extension `.txt`

**!!Caution!!** There is no safety net. Deleted files are not sent to a bin. They are gone for good. Before using `rm`, you wnat to be very sure you are not going to delete something you need. This is particulary true when using `*` to delete multiple files.

**Exercise 4:** Once you have successfully unzipped the folders use the command `rm` to remove the zip files and free up space. If you want to, you can use `*.zip` to remove all of them with one command. **Note** When using pattern matching to `rm` multiple files, it is a good idea to check what you are removing first using `ls`

In [ ]:
#run this cell to check exercise 4:
ls -d */
ls *.zip | wc -l

**Expected output:**
```
__MACOSX/	patient_data_1/	patient_data_2/	reads/
ls: *.zip: No such file or directory
       0
```

**Making, naming and moving:** the command `mkdir` makes directories. For example `mkdir dir1` will make a  subdirectory within your current working directory called `dir1` with. You can pass mkdir multiple arguments, for example `mkdir dir1 dir2 dir3` will make three directories called  `dir1`, `dir2`, and `dir3`.

`mv` is a command that moves and renames files. For example if I wanted to copy all files ending in the extension `.txt` from my current working directory to a directory one level up I would type `mv *.txt ..` where `..` refers to the directory one level up. The command `mv *.txt ../new_location` would move all txt files to a folder called `new_location` that is one level up from my current working directory. You can also use `mv` to rename files. For example, the command `mv old_name.txt new_name.txt` would rename the file `old_name.txt` to `new_name.txt`

**Exercise 5:** Using `mkdir` make a new directory in `339_lab-main` called `patient_data` then use the command we have discussed to change to move all of the files from `patient_data_1` and `patient_data_2` to this folder. 

In [ ]:
#run this cell to check exercise 5
ls patient_data_*/* | wc -l
ls patient_data/* | wc -l

**Expected output:**
```
ls: patient_data_*/*: No such file or directory
       0
      24
```

**Exercise 6:** the command `ls patient_data/* | wc -l` will count the number of files in the folder `patient_data`. We won't unpack exactly how this works right now, but you should be able to figure it out with a little more BASH learning. modify the command so that it will only count the files that end in the extension `.fq`. **Hint:** You will need to use `*.fq` somewhere in your command

**Expected output:**
```
24
```

Execute the cell below into to the directory `~/339_lab-main/reads` and list all of the files. Notice that there are files that end with `reads1.fq` and files that end with `reads2.fq` these are forward and reverse reads respectively from sequencing of ten different genomes. 

In [ ]:
cd ~/339_lab-main/reads
ls 

**Exercise 7:** write a command that will list only the forward reads. Hint: you can use the wild card `*` more than once. For example `ls *something*.txt` would list all text files with the pattern `something` in them

**Expected output:**
```
genome_10_reads1.fq	genome_4_reads1.fq	genome_8_reads1.fq
genome_1_reads1.fq	genome_5_reads1.fq	genome_9_reads1.fq
genome_2_reads1.fq	genome_6_reads1.fq
genome_3_reads1.fq	genome_7_reads1.fq
```


## Variables

BASH uses the equal sign to assign variables as shown in the cell below. Execute the commands and see what happens. 


In [ ]:
course=biol339
echo $course

**Note:** `echo` is a command that prints something to the console. 

**Note:** The dollar symbol `$` can be thought of as meaning *the value of*. So the command `echo $course` prints the value we previously assigned to the variable named course.

**Note:** In jupyter lab, variables that you assign in one cell will persist in subsequent cells. 

**Note:** You can also use `$` to place the value of your variables in sentences or commands. See the cell below for an example of this using the `echo` command.

In [ ]:
echo I love $course

## Arrays:
Below is an exerpt from **the unix work bench** by Sean Ross that provides a concise introduction to arrays. Make sure you understand all of this before moving on.

_______________________________________________________________________________________________

Arrays in Bash are ordered lists of values. You can create a list from scratch by assigning it to a variable name. Lists are created with parentheses `( )` with a space separating each element in the list. Let’s make a list of the plagues of Egypt:

```bash
plagues=(blood frogs lice flies sickness boils hail locusts darkness death)
```

To retrieve the array you need to use parameter expansion, which involves the dollar sign and curly brackets `${ }`. The positions of the elements in the array are numbered starting from zero. To get the first element of this array use `${plagues[0]}`like so:

```bash
echo ${plagues[0]}
```
`blood`

`echo` is a command that prints something to the console. the symbol `$` can be thought of as meaning **the value of** so the command above is asks the shell to print the value of the first element of the array `plagues` to the console.

**Notice** that the first element has an index of 0. You can get any of the elements this way, for example the fourth element:

```bash
echo ${plagues[3]}
```
`flies`

To get all of the elements of plagues use a star (*) between the square brackets:
```bash
echo ${plagues[*]}
```
`blood frogs lice flies sickness boils hail locusts darkness death`

You can also change an individual elements in the array by specifying their index with square brackets:

```bash
plagues[4]=disease
echo ${plagues[*]}
```

`blood frogs lice flies disease boils hail locusts darkness death`

You can find the length of an array using the pound sign `#`:

```bash
echo ${#plagues[*]}
```

`10`

**Summary**

 - Arrays are a linear data structure with ordered elements which can be stored in variables.
 - The each element of an array has an index and the first index is 0. 
 - Individual elements of an array can be accessed using their index.
_______________________________________________________________________________________________

**Note:** You can use `*` to make arrays of items matching a particular pattern. For example, the cell below will create an array called `fw_reads` that contains all file names in the current directory that contain the pattern `reads1`



In [ ]:
fw_reads=(*reads1*)

**Exercise 8:** In the cell below, write a command that will get the length of the array `fw_reads`

**Expected output:**
```
10
```

**Exercise 9:** In the cell below write commands that use `echo` to print the first, second, third and tenth element of the array `fw_reads`. Your answer should contain three lines.

**Expected output:**
```
genome_10_reads1.fq
genome_1_reads1.fq
genome_2_reads1.fq
genome_9_reads1.fq
```

## For loops
In BASH, and other programming languages, for loops allow us to iterate through items in an array. For example, the code in the cell below iterates through `fw_reads` and prints each item it contains to the console.

In [ ]:
for i in ${fw_reads[@]}
do
    echo $i
done

**Let's look at the structure of the `for` loop above one line at a time:**

The first line, `for i in ${fw_reads[@]}`, specifies the thing we are iterating. In this case we are iterating through the array `fw_reads` that we created earlier. This line also assigns a varible name for iteration. In this case the variable is called `i`, but  could have used `j` or `k` or `read` or `cheeseburger` or another word with no spaces. On the first iteration, the value of `i` will be zero. The variable `i` changes by one as we step through each item in the thing we are iterating. 

Everything between the lines **`do`** and **`done`** is a command that will be carried out at every iteration. You can put multiple commands on each line. In this case we are using the `echo` command and `$` to print the value of `i` as it changes with each iteration. 

The indentation is added for clarity of formatting. You can also make for loops on one line by using `;` as a seperator. See the cell below for an example:

In [ ]:
for i in ${fw_reads[@]}; do echo $i; done

**Note:** By adding an exlaimation mark before the array name, we can iterate through the indicies of the array as shown below.

In [ ]:
for i in ${!fw_reads[@]}
do
    echo $i
done

**Note:** We can also use the indicies to grab the corresponding item from the array as shown below. It will soon become clear why this is a useful thing to be able to do.

In [ ]:
for i in ${!fw_reads[@]}
do
    echo ${fw_reads[$i]}
done

**Exercise 10:** In the cell below, Make another array called `rv_reads` which contains the names of all of the reverse read files in the folder `reads`. Recall that these contain the pattern `reads2`

In [ ]:
#run this cell to check exercise 10
echo ${#rv_reads[*]}
echo ${rv_reads[9]}

**Expected output:**
```
10
genome_9_reads2.fq
```

**Note:** We can retrive items from the same position in two or more arrays as shown in the cell below. Note that `echo` is printing everything you put after it including the words: `and`, `are` ,`at` and `position`

In [ ]:
echo ${fw_reads[2]} and ${rv_reads[2]} are at position 2

**Exercise 11:** In the cell below, wite a for loop that iterates trhough the indicies of `fw_reads` and prints a statement of the type shown above for each element in both the `fw_reads` and `rv_reads` arrays. You will notice that the corresponding fw and rv reads occupy the same position in their respective arrays. This provides a convenient way of passing matching forward and reverse read sets to another command or to a software tool, as we will soon see...

**Expected Output:**
```
genome_10_reads1.fq and genome_10_reads2.fq are at position 0
genome_1_reads1.fq and genome_1_reads2.fq are at position 1
genome_2_reads1.fq and genome_2_reads2.fq are at position 2
genome_3_reads1.fq and genome_3_reads2.fq are at position 3
genome_4_reads1.fq and genome_4_reads2.fq are at position 4
genome_5_reads1.fq and genome_5_reads2.fq are at position 5
genome_6_reads1.fq and genome_6_reads2.fq are at position 6
genome_7_reads1.fq and genome_7_reads2.fq are at position 7
genome_8_reads1.fq and genome_8_reads2.fq are at position 8
genome_9_reads1.fq and genome_9_reads2.fq are at position 9
```

# Assembling genomes from illumina sequence reads:

Now that you have learned a little bit of BASH, you are ready to run your first bioinformatics software package, but first a little bit about  **command line applications**

Command line applications are pieces of software that are deployed using the command line. We installed a bunch of these at the start of the lab, now we are going to start using them. Command line applications can be called using their name. The can are given arguments that specificy which files they should act on, and what the should do to these files.

## Adapter removal

Before we can assemble genomes from our reads, we need to trim the reads to remove adapter sequences that were added during the sequencing run. We will do this using a program called `AdapterRemoval`. This program will identify and remove adapter sequences from our reads, and create new files from the resulting trimmed reads.

We are going to write a short script that uses a `for` loop to trim the adapters from all of our forward and reverse reads.

If we wanted to use adapter removal to trim a single pair of matching forward and reverse reads, we would use the command shown below:

```bash
AdapterRemoval --file1 fwd_reads.fq --file2 rv_reads.fq --basename new_file_name
--trimns --trimqualities --collapse
```

Whatever we put after `--file1` tells AdapterRemoval where the forward reads file is. `--file2` tells Adapter removal where the reverse read file is. `--basename` tells AdapterRemoval what name to give the trimmed read files. `--trimns` and `--trimqualities` and `--collapse` are additional arguments that tell AdapterRemoval to remove dubious data and do some other stuff.

In **exercises 12-15** we are going to build the pieces of our adapter removal script one-by-one and bring them together to make our final script for adapter removal.

**Exercise 12:** In the cell below, write a for loop that iterates through the indices of the array `fw_reads` and uses the number returned to `echo` the corresponding values from both `fw_reads` and `rv_reads` on seperate lines

**Expected output:**
```
genome_10_reads1.fq
genome_10_reads2.fq
genome_1_reads1.fq
genome_1_reads2.fq
genome_2_reads1.fq
genome_2_reads2.fq
genome_3_reads1.fq
genome_3_reads2.fq
genome_4_reads1.fq
genome_4_reads2.fq
genome_5_reads1.fq
genome_5_reads2.fq
genome_6_reads1.fq
genome_6_reads2.fq
genome_7_reads1.fq
genome_7_reads2.fq
genome_8_reads1.fq
genome_8_reads2.fq
genome_9_reads1.fq
genome_9_reads2.fq
```

**Exercise 13:** Update the your for loop so that it first assigns the values to variables called `reads_1` and `reads_2`, then uses `echo` and  `$` to print the values of these variables on the same line

**Expected output:**
```
genome_10_reads1.fq genome_10_reads2.fq
genome_1_reads1.fq genome_1_reads2.fq
genome_2_reads1.fq genome_2_reads2.fq
genome_3_reads1.fq genome_3_reads2.fq
genome_4_reads1.fq genome_4_reads2.fq
genome_5_reads1.fq genome_5_reads2.fq
genome_6_reads1.fq genome_6_reads2.fq
genome_7_reads1.fq genome_7_reads2.fq
genome_8_reads1.fq genome_8_reads2.fq
genome_9_reads1.fq genome_9_reads2.fq
```
**Run the additional check cell for exercise 13 below as well**

In [ ]:
#run this cell to check exercise 13
echo $reads_1 $reads_2

**Expected output:**
```
genome_9_reads1.fq genome_9_reads2.fq
```

**Helper Code**

*Notice* that we could pass the read file names to `AdapterRemoval` instead of `echo` using the same logic. To do this, we will also need to generate a new file name and store this as a third variable. You cut and paste the extremely nasty looking line of code below to do this.

```bash
genome_name=${reads_1%??????????} 
```
This line of code strips the last n characters from `reads_1` where the number of question marks specifies. This gives us the common prefix for both `reads1` and `reads2` and stores it as a new variable called`$genome_name`. This can now be passed to the argument `--basename` so that `AdapterRemoval` knows what prefix it should use for the new files it creates.

**Exercise 14:** Update your for loop so that it assigns an additional variable called `genome_name` as described above and prints all three variables on a single line. You can just cut and paste the line from the cell above if you want.

**Expected output:**
```
genome_10_reads1.fq genome_10_reads2.fq genome_10
genome_1_reads1.fq genome_1_reads2.fq genome_1
genome_2_reads1.fq genome_2_reads2.fq genome_2
genome_3_reads1.fq genome_3_reads2.fq genome_3
genome_4_reads1.fq genome_4_reads2.fq genome_4
genome_5_reads1.fq genome_5_reads2.fq genome_5
genome_6_reads1.fq genome_6_reads2.fq genome_6
genome_7_reads1.fq genome_7_reads2.fq genome_7
genome_8_reads1.fq genome_8_reads2.fq genome_8
genome_9_reads1.fq genome_9_reads2.fq genome_9
```
**Run the additional check cell for exercise 14 below as well**

In [ ]:
#run this cell to check exercise 14
echo $reads_1 $reads_2 $genome_name

**Expected output:**
```
genome_9_reads1.fq genome_9_reads2.fq genome_9
```

**Exercise 15:**  Update your for loop so that it passes the variables `reads_1`, `reads_2` and `genome_name` as arguments to AdapterRemoval. You should also add `--trimns --trimqualities --collapse` as shown in our previous example.

In [ ]:
#run this cell to check exercise 15
ls *pair*.truncated | wc -l

**Expected output:**
```
20
```

**Exercise 16:** If your script worked, your folder should now contain a bunch of files that end in `pair1.truncated` and `pair2.truncated` these are your trimmed forward and reverse reads. 

Once you have confirmed these are present:

 - Make a new directory `../trimmed_reads`
 - Move all the files from `reads` that match the pattern `*pair*` to your newly created directory

In [ ]:
#run this cell to check exercise 16
ls ../trimmed_reads/* | wc -l

**Expected output:**
```
20
```

**Helper Code**

Running the cell below will:

 - Remove all the extra junk that AdapterRemoval made in your reads folder.
 
 - Change directory to ../trimmed_reads
 
 - Add `.fq` as an extension to your trimmed read files so that they will be recognised by the genome assembly tool we are going to use

In [ ]:
shopt -s extglob 
rm !(*.fq)
cd ../trimmed_reads
for f in *.truncated; do mv "$f" "$f.fq"; done

In [ ]:
#Run this cell to check that the helper code worked
ls *.fq | wc -l

**Expected output:**
```
20
```

## Assembling your genomes:

You now have all the skills required to assemble your trimmed reads using `spades`. Genome assembly algorithms like the one used bu `spades` take an overlapping collection of reads and rebuild a continuos genome sequence from them. This is analogus to exploding a stack of newspapers into 150 word chunks then using the overlapping fragments of text to reconstitute the entire newspaper. Genome assembly algorithms are super interesting, but we don't have time to go into exactly how they work here. If you want to know more, let me know!

**The command for assembling a single pair of fw and rv read sets with `spades` is shown below:** 

    spades.py --pe1-1 fw_reads.fq --pe1-2 rv_reads.fq -o output_directory

**Exercise 17:** Using the same approach you applied for adapter removal, write a script containing a `for` loop that uses `*` to make arrays of fw and rv trimmed reads and iterates through indicies to pass matching fw and rv read sets to SPAdes.

**Note:** You will also need to use mkdir once in every iteration to make a new directory to send the assembly outputs to. You should name your new directories using the current index of the `for` loop. If the index is called `i`, as in our previous example, you'd make the directory with this command: `mkdir $i`, and pass the directory name to spades as argument using  `-o $i` so that SPAdes sends the outputs from each iteration of the loop to a newly created and uniquely named directory. 

**Note:** `spades` will take a minute or so to process each genome, so this exercise will take a few minutes to complete. I have noticed that running spades will often freeze the notebook. If the check cell still hasn't output anything after 10 min, restart the kernel and run the check cell again. If it gives the expected output of `10` you are good to go. 

In [ ]:
#run this cell to get back to the right directory after restarting the kernel
cd ~/339_lab-main/trimmed_reads

In [ ]:
#run this cell to check exercise 17
cd ~/339_lab-main/trimmed_reads
ls -d [0-9]/contigs.fasta | wc -l

**Expected output:**
```
10
```

**Helper Code**

Running the code provided below will:

 - Make a new directory `../assemblies`
 
 - Copy each of your newly assembled genomes to this directory and give it a unique name
 
 - Print some data about the sizes and coverages of the assembled genomes

In [ ]:
mkdir ../assemblies
fw_reads=(*pair1*)
for i in ${!fw_reads[@]}
do
    echo $i
    file_name=${fw_reads[$i]} 
    new_name="${file_name%%.*}.fna"
    echo $new_name
    contig_file=$i/contigs.fasta
    head -n 1 $contig_file
    cp $contig_file ../assemblies/$new_name
    
done

**Expected output:**
```
0
genome_1.fna
>NODE_1_length_65499_cov_24.041806
1
genome_10.fna
>NODE_1_length_69578_cov_24.027899
2
genome_2.fna
>NODE_1_length_70716_cov_23.833194
3
genome_3.fna
>NODE_1_length_70951_cov_23.952253
4
genome_4.fna
>NODE_1_length_71348_cov_23.934532
5
genome_5.fna
>NODE_1_length_71099_cov_24.003210
6
genome_6.fna
>NODE_1_length_71297_cov_23.971525
7
genome_7.fna
>NODE_1_length_71428_cov_23.952110
8
genome_8.fna
>NODE_1_length_71188_cov_23.939278
9
genome_9.fna
>NODE_1_length_74511_cov_23.967945
```

# Annotating your genome assemblies using glimmer3

We will now identify genes within each of the newly assembled genomes using another program caleld `glimmer3`. First, lets change to the directory where we stored your assembled genomes, and see what is in there

In [ ]:
cd ../assemblies
ls

**Expected output:**
```
genome_1.fna	genome_2.fna	genome_4.fna	genome_6.fna	genome_8.fna
genome_10.fna	genome_3.fna	genome_5.fna	genome_7.fna	genome_9.fna
```

**A brief note on how to use glimmer3:** `glimmer3` requires that a model be built for each genome before it can find genes. This is achieved using three helper programs called `long-orfs`, `extract` and `build-icm`. The `extract` program can also be used to pull out the gene sequences that `glimmer3` identifies and store these as a new file. The example script below To annotates a single genome using `glimmer3`. This script assumes that the file name for the genome to be annotated has previuosly been assigned as a variable called `genome_file`, such that the file name can be accessed using `$genome_file`

```bash
long-orfs $genome_file.fna long_orfs.txt
extract -t $genome_file.fna long_orfs.txt > run1.train
build-icm -r run1.icm < run1.train
glimmer3 $genome_file.fna run1.icm run1
extract -t $genome_file.fna run1.predict > genes_$genome_file.fna
rm run1* long_orfs.txt 
```
**The first line**, `long-orfs genome_file.fna long_orfs.txt`, runs the program `long-orfs` to identify long open reading frames (ORFs) in the specified genome assembly `genome_file.fna`

**The second line**, `extract -t genome_file.fna long_orfs.txt > run1.train`, runs the program `extract` to extract the long orf sequences identified in line one and store these in a new file called `run1.train`

**the third line**, `build-icm -r run1.icm < run1.train`, uses the program `build-icm` to build a gene finding model for the genome using the training data `run1.train` generated in line two. The model is stored as a file called `run1.icm`

**the fourth line**, `glimmer3 $genome run1.icm run1`, runs `glimmer3` on `genome_file.fna` and uses the model `run1.icm` to find the coordinates of genes in `genome_file.fna`, these are stored in a file called `run1.predict`

**the fifth line**, `extract -t genome_file.fna run1.predict > genes_$genome`, uses `extract` again to pull out the sequence of the genes according to the coordiates in `run1.predict` and store these in a new file called `genes_genome_file.fna`

**the last line** `rm run1* long_orfs.txt` removes the files that are no longer needed.

**Exercise 18:** Use what you have already learned, and the example above, to write a genome annotation script. Your script should make an array by matching all files in the `assemblies` directory that end with the extension `.fna`. It should then use a `for` loop to iterate through the resulting array of file names, and extract the gene sequences from each genome. You will need to store these as uniquely named files by adding the prefix `genes_` to each genome file name so that you end up with files called `genes_genome_1.fna`, `genes_genome_2.fna` ... `genes_genome_10.fna ` It might be helpful to test pieces of the script before putting the whole thing together. **Note:** If all has gone well, you should already be in the directory 
`~/339_lab-main/assemblies`

In [ ]:
#run this cell to check exercise 18
wc -l genes_genome* 

**Expected output:**
```
    1065 genes_genome_1.fna
    1132 genes_genome_10.fna
    1105 genes_genome_2.fna
    1177 genes_genome_3.fna
    1111 genes_genome_4.fna
    1153 genes_genome_5.fna
    1098 genes_genome_6.fna
    1112 genes_genome_7.fna
    1120 genes_genome_8.fna
    1219 genes_genome_9.fna
   11292 total
```

**Helper Code**

Running the cell below will combine the extracted genes from all of your genomes into a single file called `all_genes.fna` that will be used for mapping the RNAseq data in the `patient_data` folder

In [ ]:
for f in genes_*.fna; do sed -i -e "s/^>/>${f%.fna}_/g" "${f}"; done
rm *fna-e
cat genes* > all_genes.fna
wc -l all_genes.fna

**Expected output:**
```
11292 all_genes.fna
```

# Measuring Gene Expression in the Gut Microbiome with RNAseq

The folder `patient_data` contains metatranscriptome data from the gut metagenomes of 12 patients. There are two files for each patient. These are fw and rv illumina sequencing reads from and RNAseq experiment. Six of the patients are healthy controls while the other six developed colorectal cancer within 2 years of date that the metatranscriptome data was collected. 

**What exactly is a metatranscriptome?** A transcriptome is the collection of genes that are being transcribed by a single organism, tissue sample or cell culture. Most commomnly this is measured using RNAseq. A *meta*transcriptome is the transcriptome of a collection of organisms. In our case, the data are transcriptomes from all of the gut microbes for 12 patients that were collected by extracting RNA directly from metagenome samples. Thankfully I didn't actually have to do this since the data are imaginary.

In the previous sections, we assembled the genomes for all known gut microbes in our imaginary world. We then annotated genes within these, and extracted the sequences for all of the genes, from all of the bacteria, into a single file. We are now going to use a program called `kallisto` to align our RNAseq reads against our extracted gene sequences. The output we generate for each patient dataset will be a count table where the number of transcripts for each gene are counted. We will then feed these count tables to a statistical analysis package in order to identify genes which are differentially expressed in the gut microbiomes of patients who went on to develop colorectal cancer, as compared to healthy controls. 

## Making an index file:

Before aligning RNAseq reads against our extracted gene sequences, `kallisto` needs to make an index file. The command for doing this on a file called `gene_file.fna` is shown below. 

`kallisto index -i kallisto_index gene_file.fna --make-unique`

This command is telling `kallisto` to run the `index` making algorithm to generate an index file called `kallisto_index` from the input `gene_file.fna` the flag `--make-unique` tells `kallisto` to make unique names if it encounters any duplicated gene names in the file `gene_file.fna`

**Exercise 19:** In the next cell, run this same command on the concatenated gene file you made previously.

In [ ]:
#run this cell to check exercise 19
wc -l kallisto_index

**Expected output:**
```
3027 kallisto_index
```

**Exercise 20:** change directory to `patient_data` and list the files contained in this directory

**Expected output:**
```
patient_10_1.fq	patient_12_2.fq	patient_3_1.fq	patient_5_2.fq	patient_8_1.fq
patient_10_2.fq	patient_1_1.fq	patient_3_2.fq	patient_6_1.fq	patient_8_2.fq
patient_11_1.fq	patient_1_2.fq	patient_4_1.fq	patient_6_2.fq	patient_9_1.fq
patient_11_2.fq	patient_2_1.fq	patient_4_2.fq	patient_7_1.fq	patient_9_2.fq
patient_12_1.fq	patient_2_2.fq	patient_5_1.fq	patient_7_2.fq
```

**Notice** that all of the fw read files end in 1 and all of the rv read files end in 2. 

## Aligning RNAseq reads against an index file

The index file we made is a special data structure that allows `kallisto` to rapidly align sequence reads agaisnt the sequences represented in the index. In our case, the index contains all the genes we extracted from our gut bacterial genome assemblies. In exercises 21 and 22, you will use the `quant` function of `kallisto` toalign the RNAseq data in the `patient_data` folder against your index file. This will generate a count table for each patient that keeps track of the number of RNAseq reads aligning to each of the genes in the index file. 

**Exercise 21:** In the cell bewlow, make arrays called `fw_reads` and `rv_reads` that contain the file names for all of the forward and reverse reads for each patient.

In [ ]:
#run this cell to check exercise 21
echo ${#fw_reads[*]}
echo ${fw_reads[9]}
echo ${#rv_reads[*]}
echo ${rv_reads[9]}

**Expected output:**
```
12
patient_7_1.fq
12
patient_7_2.fq
```

Execute the cell below to make the folder where we will save the outputs from `kallisto`

In [ ]:
mkdir ~/339_lab-main/outputs

The command for running `kallisto` with a single pair of RNAseq read sets is shown below:

```bash
kallisto quant -i ../assemblies/kallisto_index -o $out_dir $fw_file $rv_file
```
**Notes:**

- This command assumes that the names of the forward and reverse read files have been assigned to the variables `fw_file` and `rv_file` respectively. 

- It is telling `kallisto` to run the `quant` algorithm to map the reads in `$fw_file` and `$rv_file` against the index file `../assemblies/kallisto_index` that we created earlier. 

- The outputs will be stored in `$out_dir`, assuming this variable has also been set to an existing directory where we want to store the ouptuts from `kallisto`

**Exercise 22** In the cell below, write a script that uses a `for` loop to iterate through the fw and rv reads in all of your patent datasets and runs `kallisto quant` on these. At each iteration you should assign a variable for your fw and rv read set using the index `$i` generated by iterating trhough one of your arrays. You should also assign a variable for the output directory, make this directory, and tell `kallisto` to store the outputs there. Assuming you call your fw read file `fw_file` for each iteration, you could achieve this using the two lines shown below to make a new subdirectory `../outputs/$fw_file`.

```bash
out_dir=../outputs/$fw_file
mkdir $out_dir
```

In [ ]:
#run this cell to check the output of exercise 22
ls -d ~/339_lab-main/outputs/* | wc -l
ls ~/339_lab-main/outputs/*/*.tsv | wc -l
wc -l ~/339_lab-main/outputs/*/*.tsv

**Expected output:**
```
12
      12
     652 /Users/owenje/339_lab-main/outputs/patient_10_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_11_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_12_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_1_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_2_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_3_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_4_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_5_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_6_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_7_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_8_1.fq/abundance.tsv
     652 /Users/owenje/339_lab-main/outputs/patient_9_1.fq/abundance.tsv
    7824 total
```